In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
from typing import Dict, List, Optional
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import mlflow 
from mlflow_utils import create_dataset

load_dotenv()

True

In [2]:
def get_classification_metrics(
    y_true: pd.Series, y_pred: pd.Series, prefix: str) -> Dict[str, float]:
    """
    Get the classification metrics.

    :param y_true: The true target values.
    :param y_pred: The predicted target values.
    :param prefix: The prefix of the metric names.
    :return: The classification metrics.
    """

    return {
        f"{prefix}_accuracy": accuracy_score(y_true=y_true, y_pred=y_pred),
        f"{prefix}_precision": precision_score(y_true=y_true, y_pred=y_pred),
        f"{prefix}_recall": recall_score(y_true=y_true, y_pred=y_pred),
        f"{prefix}_f1": f1_score(y_true=y_true, y_pred=y_pred),
    }


def get_sklearn_pipeline(
    numerical_features: List[str], categorical_features: Optional[List[str]] = []) -> Pipeline:
    """
    Get the sklearn pipeline.

    :param numerical_features: The numerical features.
    :param categorical_features: The categorical features.
    :return: The sklearn pipeline.
    """

    preprocessing = ColumnTransformer(
        transformers=[
            ("numerical", SimpleImputer(strategy="median"), numerical_features),
            ("categorical", OneHotEncoder(), categorical_features),
        ]
    )

    pipeline = Pipeline(
        steps=[
            ("preprocessing", preprocessing),
            ("model", RandomForestClassifier())
        ]
    )

    return pipeline


In [3]:
# conectar con mlflow y minio (por http)
mlflow.set_tracking_uri("http://127.0.0.1:5000")

os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://127.0.0.1:9000"
os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('ACCESS_KEY')

# Pipeline



In [4]:
df = create_dataset()

In [5]:
x_train, x_test, y_train, y_test = train_test_split(
    df.drop("target", axis=1),
    df["target"],
    test_size=0.2,
    random_state=42,
)

In [6]:
numerical_features = [f for f in x_train.columns if f.startswith("feature")]
print(numerical_features)

['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20', 'feature_21', 'feature_22', 'feature_23', 'feature_24', 'feature_25', 'feature_26', 'feature_27', 'feature_28', 'feature_29', 'feature_30', 'feature_31', 'feature_32', 'feature_33', 'feature_34', 'feature_35', 'feature_36', 'feature_37', 'feature_38', 'feature_39', 'feature_40', 'feature_41', 'feature_42', 'feature_43', 'feature_44', 'feature_45', 'feature_46', 'feature_47', 'feature_48', 'feature_49']


In [7]:
pipeline = get_sklearn_pipeline(numerical_features=numerical_features)
pipeline.fit(x_train, y_train)
pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numerical',
                                                  SimpleImputer(strategy='median'),
                                                  ['feature_0', 'feature_1',
                                                   'feature_2', 'feature_3',
                                                   'feature_4', 'feature_5',
                                                   'feature_6', 'feature_7',
                                                   'feature_8', 'feature_9',
                                                   'feature_10', 'feature_11',
                                                   'feature_12', 'feature_13',
                                                   'feature_14', 'feature_15',
                                                   'feature_16', 'feature_17',
                                                   'feature_18', 'feature_19',
                                                   'feature_20', 'feature_21',
                                                   'feature_22', 'feature_23',
                                                   'feature_24', 'feature_25',
                                                   'feature_26', 'feature_27',
                                                   'feature_28', 'feature_29', ...]),
                                                 ('categorical',
                                                  OneHotEncoder(), [])])),
                ('model', RandomForestClassifier())])

In [33]:
pipeline.get_params()

{'memory': None,
 'steps': [('preprocessing',
   ColumnTransformer(transformers=[('numerical', SimpleImputer(strategy='median'),
                                    ['feature_0', 'feature_1', 'feature_2',
                                     'feature_3', 'feature_4', 'feature_5',
                                     'feature_6', 'feature_7', 'feature_8',
                                     'feature_9', 'feature_10', 'feature_11',
                                     'feature_12', 'feature_13', 'feature_14',
                                     'feature_15', 'feature_16', 'feature_17',
                                     'feature_18', 'feature_19', 'feature_20',
                                     'feature_21', 'feature_22', 'feature_23',
                                     'feature_24', 'feature_25', 'feature_26',
                                     'feature_27', 'feature_28', 'feature_29', ...]),
                                   ('categorical', OneHotEncoder(), [])])),
  ('mode

In [8]:
y_pred = pipeline.predict(x_test)
metrics = get_classification_metrics(y_true=y_test, y_pred=y_pred, prefix="test")
# Falta plots

In [9]:
mlflow.set_experiment("mlflow_tracking")

<Experiment: artifact_location='s3://mlflow/3', creation_time=1718807838682, experiment_id='3', last_update_time=1718807838682, lifecycle_stage='active', name='mlflow_tracking', tags={}>

In [22]:
with mlflow.start_run(run_name="logging_artifacts25") as run:

    # log metrics
    mlflow.log_metrics(metrics)

    # log params
    mlflow.log_params(pipeline.get_params())

    tags = {
        "type": "classifier",
        "mlflow.note.content": "This is a classifier for the house pricing dataset",
    }
    mlflow.set_tags(tags)
    
    # # log tags
    # mlflow.set_tags({"type": "classifier"})

    # # log description
    # mlflow.set_tag({'mlflow.note.content': 'This is a classifier for the house pricing dataset'})

    # log plots (faltan)

In [23]:
print('tracking uri:', mlflow.get_tracking_uri())
print('artifact uri:', mlflow.get_artifact_uri())

tracking uri: http://127.0.0.1:5000
artifact uri: s3://mlflow/3/aaff3887ad2b43efb3954de22eb1b85f/artifacts


In [24]:
mlflow.end_run()

In [25]:
print('ok_')

ok_
